In [1]:
from typing import Any

import lightning.pytorch as pl
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow
from torch.utils.data import DataLoader, Dataset

from src.ml.data.splitting import create_data_splits
from src.ml.modeling import (
    model_factory,
    optimizer_factory,
)
from src.ml.utils.set_seed import set_seed

from datetime import datetime
from loguru import logger
from pathlib import Path
from shutil import copy, copytree, rmtree

import yaml

from src.ml.train_neural_network import train_neural_network
from src.ml.data import data_sets_factory
from src.ml.preprocessing import preprocessing_factory
from src.ml.utils.set_seed import set_seed

In [2]:
CONFIG_FILE = Path("src/ml/config.yaml")

In [3]:
torch.set_default_device(torch.device("cpu"))

In [4]:
# load config file

logger.info("Loading config file.")

with open(CONFIG_FILE, "r") as f:
    config = yaml.safe_load(f)

2024-12-15 12:00:02.889 | INFO     | __main__:<module>:3 - Loading config file.


In [5]:
# load data

logger.info("Loading data.")

_, data_sets = next(iter(data_sets_factory(**config["data_set"])))

# preprocess data

logger.info("Start preprocessing.")

for preprocessing_step in config["preprocessing"]:
    logger.info("Perform {} preprocessing.", preprocessing_step["name"])

    transform = preprocessing_factory(**preprocessing_step)
    data_sets = transform(data_sets)

dataset = data_sets

2024-12-15 12:00:03.297 | INFO     | __main__:<module>:3 - Loading data.
2024-12-15 12:00:10.981 | INFO     | __main__:<module>:9 - Start preprocessing.
2024-12-15 12:00:10.982 | INFO     | __main__:<module>:12 - Perform add_taxa_names preprocessing.
2024-12-15 12:00:12.281 | INFO     | __main__:<module>:12 - Perform add_clade_information preprocessing.
2024-12-15 12:00:12.791 | INFO     | __main__:<module>:12 - Perform remove_tree preprocessing.


In [6]:
train_dataset, val_dataset, test_dataset = create_data_splits(
    dataset, **config["training"]["splitting_config"]
)

In [12]:
train_loader = DataLoader(train_dataset, **config["training"]["dataloader_config"])
test_loader = DataLoader(test_dataset,batch_size=2)
val_loader = DataLoader(val_dataset)

optimizer = optimizer_factory(**config["training"]["optimizer_config"])
model = model_factory(
    optimizer=optimizer,
    input_example=train_dataset[0],
    **config["training"]["model_config"],
)

In [13]:
from src.ml.modeling.conditional_tree_flow import ConditionalTreeFlow

model = ConditionalTreeFlow.load_from_checkpoint("ml_data/models/debug_dummy_2024_12_15_02_08_33/yule-10_140/epoch=19-val_loss=-41.34.ckpt")
model = model.eval()

In [14]:
i = iter(test_loader)

In [15]:
sample = next(i)

In [16]:
model.forward(sample)["z"] 

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan]], grad_fn=<AddBackward0>)

In [23]:
prior = model.prior.sample((2, 9,))
latent = model.forward(sample)
latent["z"] = prior.clone()
model.inverse(latent)["branch_lengths"]

tensor([[0.8971, 4.8311, 1.3888, 1.2148, 1.2583, 0.3159, 1.0022, 8.7158, 3.3843],
        [1.0496, 4.2970, 5.8533, 1.1094, 0.4983, 1.0842, 2.8284, 0.6380, 2.7224]],
       grad_fn=<ExpBackward0>)

In [14]:
transformed = latent

In [15]:
transformed["z"]

tensor([[  1.0683,   5.3490, 168.6009,   3.1450,   0.3396,   2.7504,   1.9404,
           0.2198,   4.1802]], grad_fn=<ExpBackward0>)

In [16]:
model.flows[-33]

IndexError: index -33 is out of range

In [ ]:
transformed["z"] = result["z"]
result = model.flows[-33].inverse(**transformed)

result["z"]

tensor([[2.2004e-01, 8.4510e+02, 3.1984e+01, 0.0000e+00, 5.3590e-03, 3.6059e-01,
         8.8577e+04, 1.3899e+22, 3.2386e-04]], grad_fn=<ExpBackward0>)

In [ ]:
output["z"], output["branch_lengths"]

(tensor([[4.5399e+01, 9.2058e-05,        inf, 1.1351e+17, 9.7374e+00, 2.7609e+01,
          3.2553e-15, 5.0594e+25, 1.2663e+03]], grad_fn=<ExpBackward0>),
 tensor([[4.5399e+01, 9.2058e-05,        inf, 1.1351e+17, 9.7374e+00, 2.7609e+01,
          3.2553e-15, 5.0594e+25, 1.2663e+03]], grad_fn=<ExpBackward0>))

In [ ]:
from torch import tensor


clade_bitstrings = [
    tensor([3]),
    tensor([12]),
    tensor([19]),
    tensor([31]),
    tensor([96]),
    tensor([224]),
    tensor([255]),
    tensor([768]),
    tensor([1023]),
]

In [ ]:
clade_bitstrings = torch.tensor(clade_bitstrings)

In [ ]:
import torch

mask = 2**torch.arange(10)

In [ ]:
clade_bitstrings.unsqueeze(-1).bitwise_and(mask).ne(0).byte()

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.uint8)